In [1]:
import numpy as np
import pandas as pd
import json
import sqlite3
# import datasets as ds
# import transformers as tr

In [2]:
!mkdir data
!curl -L https://huggingface.co/datasets/NumbersStation/NSText2SQL/resolve/main/train.jsonl > data/number_station_full.jsonl

mkdir: cannot create directory ‘data’: File exists


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1121  100  1121    0     0   4953      0 --:--:-- --:--:-- --:--:--  4960
100  412M  100  412M    0     0  8447k      0  0:00:50  0:00:50 --:--:-- 8128k


In [3]:
def valid_sql(query: str) -> bool:
    with sqlite3.connect(":memory:") as conn:
        try:
            conn.executescript(query)
        except (sqlite3.OperationalError, sqlite3.IntegrityError) as e:
            return False
    return True

def remove_nlp_inst(inst: str) -> str:
    return ("\n".join(inst.split('\n')[:-6])).replace('\n\n', ';\n\n') + ";\n\n"

In [4]:
with open("data/number_station_full.jsonl", 'r') as f:
    c = f.readlines()
data = list(map(lambda point: json.loads(point), c))
df = pd.DataFrame(data)
# df = df.sample(1000)
df.head(4)

,instruction,output,source
0,"CREATE TABLE Comments (\n Id number,\n P...","SELECT Id AS ""post_link"", * FROM Posts JOIN Po...",sede
1,"CREATE TABLE table_203_815 (\n id number,\n...","SELECT ""player"" FROM table_203_815 WHERE ""coll...",squall
2,CREATE TABLE t_kc24 (\n ACCOUNT_DASH_DATE t...,SELECT COUNT(*) FROM t_kc21 JOIN t_kc22 JOIN t...,css
3,"CREATE TABLE table_17232 (\n ""Disc"" real,\n...","SELECT MIN(""Track"") FROM table_17232",wikisql
4,"CREATE TABLE course (\n crs_code text,\n ...","SELECT T2.emp_fname, T4.prof_office, T3.crs_de...",spider
...,...,...,...
289283,"CREATE TABLE Comments (\n Id number,\n P...","SELECT Id AS ""post_link"", Tags, ViewCount, Cre...",sede
289284,"CREATE TABLE table_24485 (\n ""Day"" text,\n ...","SELECT ""Time"" FROM table_24485 WHERE ""Stage"" =...",wikisql
289285,CREATE TABLE demographic (\n subject_id tex...,SELECT COUNT(DISTINCT demographic.subject_id) ...,mimicsql_data
289286,CREATE TABLE table_18598175_2 (\n method_of...,SELECT method_of_elimination FROM table_185981...,sql_create_context


In [5]:
df['sql_setup'] = df['instruction'].apply(remove_nlp_inst)
df['output'] = df['output'] + ';'
df['total_sql'] = df['sql_setup'] + df['output']

row = df.iloc[0]
# print(row['sql_setup'])
# print('------------------------')
print(row['total_sql'])
# df.head(5)

CREATE TABLE Comments (
    Id number,
    PostId number,
    Score number,
    Text text,
    CreationDate time,
    UserDisplayName text,
    UserId number,
    ContentLicense text
);

CREATE TABLE ReviewRejectionReasons (
    Id number,
    Name text,
    Description text,
    PostTypeId number
);

CREATE TABLE Badges (
    Id number,
    UserId number,
    Name text,
    Date time,
    Class number,
    TagBased boolean
);

CREATE TABLE Users (
    Id number,
    Reputation number,
    CreationDate time,
    DisplayName text,
    LastAccessDate time,
    WebsiteUrl text,
    Location text,
    AboutMe text,
    Views number,
    UpVotes number,
    DownVotes number,
    ProfileImageUrl text,
    EmailHash text,
    AccountId number
);

CREATE TABLE TagSynonyms (
    Id number,
    SourceTagName text,
    TargetTagName text,
    CreationDate time,
    OwnerUserId number,
    AutoRenameCount number,
    LastAutoRename time,
    Score number,
    ApprovedByUserId number,
    ApprovalD

,instruction,output,source,sql_setup,total_sql
0,"CREATE TABLE Comments (\n Id number,\n P...","SELECT Id AS ""post_link"", * FROM Posts JOIN Po...",sede,"CREATE TABLE Comments (\n Id number,\n P...","CREATE TABLE Comments (\n Id number,\n P..."
1,"CREATE TABLE table_203_815 (\n id number,\n...","SELECT ""player"" FROM table_203_815 WHERE ""coll...",squall,"CREATE TABLE table_203_815 (\n id number,\n...","CREATE TABLE table_203_815 (\n id number,\n..."
2,CREATE TABLE t_kc24 (\n ACCOUNT_DASH_DATE t...,SELECT COUNT(*) FROM t_kc21 JOIN t_kc22 JOIN t...,css,CREATE TABLE t_kc24 (\n ACCOUNT_DASH_DATE t...,CREATE TABLE t_kc24 (\n ACCOUNT_DASH_DATE t...
3,"CREATE TABLE table_17232 (\n ""Disc"" real,\n...","SELECT MIN(""Track"") FROM table_17232;",wikisql,"CREATE TABLE table_17232 (\n ""Disc"" real,\n...","CREATE TABLE table_17232 (\n ""Disc"" real,\n..."
4,"CREATE TABLE course (\n crs_code text,\n ...","SELECT T2.emp_fname, T4.prof_office, T3.crs_de...",spider,"CREATE TABLE course (\n crs_code text,\n ...","CREATE TABLE course (\n crs_code text,\n ..."


In [6]:
df['setup_valid'] = df['sql_setup'].map(valid_sql)

In [7]:
df['all_valid'] = df['total_sql'].map(valid_sql)

In [8]:
df['inst_valid'] = df['instruction'].map(valid_sql)

In [9]:
print(df.iloc[0]['total_sql'])

CREATE TABLE Comments (
    Id number,
    PostId number,
    Score number,
    Text text,
    CreationDate time,
    UserDisplayName text,
    UserId number,
    ContentLicense text
);

CREATE TABLE ReviewRejectionReasons (
    Id number,
    Name text,
    Description text,
    PostTypeId number
);

CREATE TABLE Badges (
    Id number,
    UserId number,
    Name text,
    Date time,
    Class number,
    TagBased boolean
);

CREATE TABLE Users (
    Id number,
    Reputation number,
    CreationDate time,
    DisplayName text,
    LastAccessDate time,
    WebsiteUrl text,
    Location text,
    AboutMe text,
    Views number,
    UpVotes number,
    DownVotes number,
    ProfileImageUrl text,
    EmailHash text,
    AccountId number
);

CREATE TABLE TagSynonyms (
    Id number,
    SourceTagName text,
    TargetTagName text,
    CreationDate time,
    OwnerUserId number,
    AutoRenameCount number,
    LastAutoRename time,
    Score number,
    ApprovedByUserId number,
    ApprovalD

In [10]:
df['inst_valid'].value_counts()

inst_valid
True     169151
False    120137
Name: count, dtype: int64

In [11]:
df['setup_valid'].value_counts()

setup_valid
True     288397
False       891
Name: count, dtype: int64

In [12]:
df['all_valid'].value_counts()

all_valid
True     271552
False     17736
Name: count, dtype: int64

In [13]:
def extract_prompt(inst):
    return "\n".join(inst.split('\n')[-5:])
df['prompt_only'] = df['instruction'].apply(extract_prompt)

In [14]:
print('--')
print(df.iloc[0]['prompt_only'])
print('--')

--

-- Using valid SQLite, answer the following questions for the tables provided above.

-- JanitorTags: tag:Linux but focused on a distro (pacman).

--


In [15]:
df['instruction'] = df['sql_setup'] + "" +  df['prompt_only']

In [16]:
print(df.iloc[0]['instruction'])

CREATE TABLE Comments (
    Id number,
    PostId number,
    Score number,
    Text text,
    CreationDate time,
    UserDisplayName text,
    UserId number,
    ContentLicense text
);

CREATE TABLE ReviewRejectionReasons (
    Id number,
    Name text,
    Description text,
    PostTypeId number
);

CREATE TABLE Badges (
    Id number,
    UserId number,
    Name text,
    Date time,
    Class number,
    TagBased boolean
);

CREATE TABLE Users (
    Id number,
    Reputation number,
    CreationDate time,
    DisplayName text,
    LastAccessDate time,
    WebsiteUrl text,
    Location text,
    AboutMe text,
    Views number,
    UpVotes number,
    DownVotes number,
    ProfileImageUrl text,
    EmailHash text,
    AccountId number
);

CREATE TABLE TagSynonyms (
    Id number,
    SourceTagName text,
    TargetTagName text,
    CreationDate time,
    OwnerUserId number,
    AutoRenameCount number,
    LastAutoRename time,
    Score number,
    ApprovedByUserId number,
    ApprovalD

In [17]:
df[['instruction', 'output', 'source', 'setup_valid', 'all_valid']].to_pickle("data/sql_executed.pkl")